In [ ]:
#install requirements
%pip install lifelines

In [ ]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times
from lifelines.statistics import pairwise_logrank_test
from lifelines import datasets
from google.colab import auth, drive


In [ ]:
#load the dataset into a dataframe
df = datasets.load_stanford_heart_transplants()

In [ ]:
#inspect data
df.head()



##Test assumption of proportional hazards for surgery vs non-surgery


In [ ]:
#create time-dependent interaction term
df['time*surgery'] = df['surgery'] * df['stop']

#create duration variable
df['duration'] = df['stop']-df['start']

In [ ]:
# Fit the Kaplan-Meier model
kmf = KaplanMeierFitter()

# Helper function for plotting Kaplan-Meier curves at the covariate level
def plot_km(col):
  ax = plt.subplot(111)
  for r in df[col].unique():
    ix = df[col] == r
    kmf.fit(durations=df['stop'][ix],
            event_observed=df['event'][ix],
            entry=df['start'][ix],
            label=r)
    kmf.plot(ax=ax, title=col)

plot_km('surgery')

In [ ]:
#Log-Log Plots

# Fit the Kaplan-Meier model
kmf = KaplanMeierFitter()

# Utility function for plotting
def plot_km_loglog(col):
  ax = plt.subplot(111)
  for r in df[col].unique():
    ix = df[col] == r
    #print(ix)
    kmf.fit(durations=df['stop'][ix],
            event_observed=df['event'][ix],
            entry=df['start'][ix],
            label=r)
    kmf.plot_loglogs(ax=ax, title=col)

plot_km_loglog('surgery')


In [ ]:
#Time Varying Interaction

#Import coxtimevarying
from lifelines import CoxTimeVaryingFitter

#Instantiate the ctv class
ctv = CoxTimeVaryingFitter()

#Fit the model
ctv.fit(df,
        id_col = None, #only use with multiple measures of a person
        event_col = 'event',
        start_col = 'start',
        stop_col = 'stop')

ctv.print_summary()

In [ ]:
#Schoenfeld's residuals

#import CPHFitter
from lifelines import CoxPHFitter

#instantiate a cph class
cph = CoxPHFitter()

#Call fit
##https://github.com/CamDavidsonPilon/lifelines/blob/716b20441768ad399e40ff71baad8fea872c43e5/lifelines/fitters/coxph_fitter.py#L167C13-L167C13
cph.fit(df[['age', 'surgery','duration','event']],
        duration_col='duration',
        event_col='event')


#Check assumptions
cph.check_assumptions(df[['age', 'surgery','duration','event']], p_value_threshold=0.05, show_plots=True)


In [ ]:
#What happens if we don't specify plots in the output?
cph.check_assumptions(df[['age', 'surgery','duration','event']], p_value_threshold=0.05)


In [ ]:
cph.print_summary()

In [ ]:
cph.plot(hazard_ratios=True)